## MOVIE RECOMMENDER SYSTEM

## Introduction

This project focuses on building a movie recommender system based on the Movielens dataset. It consists of 100,000 ratings by 610 unique users for 9724 unique movies. The objective is to build a model that can recomend top  5  movies to the users based on their ratings using collaborative filtering. This project addresses the business problem of personalized movie recommendations using the MovieLens dataset.
The dataset contains user ratings, movie metadata (title, genres, IDs), and tags, making it well-suited for collaborative filtering approaches since it captures both user behavior and movie attributes. The final dataset retained userId, movieId, title, genres, rating, and rating year, which represent the most important features for capturing user preferences over time. 

For data preparation, we dropped duplicate identifiers, merged multiple files (ratings, movies, links, tags), and imputed missing values with zeroes to ensure all users and movies were represented in the interaction matrix. We also applied scaling for model stability. Data preparation was handled with pandas (merging/cleaning), scikit-learn (SimpleImputer, StandardScaler, Pipeline), and numpy (matrix operations).

Modeling was performed using collaborative filtering via matrix factorization. Specifically, we built a Pipeline with TruncatedSVD for dimensionality reduction and Ridge Regression as the estimator. Hyperparameters were tuned with GridSearchCV, enabling cross-validated selection of latent factors and regularization strength.Evaluation was conducted with train-test split and 5-fold cross-validation. The best model achieved strong predictive performance, with RMSE of 3.29. 

## Import  libraries

In [ ]:
# Import  libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import Ridge
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, make_scorer
import warnings
warnings.filterwarnings('ignore')


## Load and merge datasets

In [2]:
# Load and display the first few rows of the dataset
df_links=pd.read_csv('ml-latest-small\links.csv')
df_links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [3]:
#Load and display the first few rows of the movies dataset
df_movies=pd.read_csv('ml-latest-small\movies.csv')
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
#Load and display the first few rows of the ratings dataset
df_ratings=pd.read_csv(r'ml-latest-small\\ratings.csv')
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
#Load and display the first few rows of the tags dataset
df_tags=pd.read_csv(r'ml-latest-small\\tags.csv')
df_tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [8]:
#Merge all datasets into a single dataframe
ratings_tags = pd.merge(df_ratings, df_tags, on=["userId", "movieId"], how="left")
ratings_tags_movies = pd.merge(ratings_tags, df_movies, on="movieId", how="left")
df_merged = pd.merge(ratings_tags_movies, df_links, on="movieId", how="left")
df_merged.head()


,userId,movieId,rating,timestamp_x,tag,timestamp_y,title,genres,imdbId,tmdbId
0,1,1,4.0,964982703,NaN,NaN,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0
1,1,3,4.0,964981247,NaN,NaN,Grumpier Old Men (1995),Comedy|Romance,113228,15602.0
2,1,6,4.0,964982224,NaN,NaN,Heat (1995),Action|Crime|Thriller,113277,949.0
3,1,47,5.0,964983815,NaN,NaN,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,114369,807.0
4,1,50,5.0,964982931,NaN,NaN,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,114814,629.0


In [7]:
# Display information about the final dataframe
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102677 entries, 0 to 102676
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   userId       102677 non-null  int64  
 1   movieId      102677 non-null  int64  
 2   rating       102677 non-null  float64
 3   timestamp_x  102677 non-null  int64  
 4   tag          3476 non-null    object 
 5   timestamp_y  3476 non-null    float64
 6   title        102677 non-null  object 
 7   genres       102677 non-null  object 
 8   imdbId       102677 non-null  int64  
 9   tmdbId       102664 non-null  float64
dtypes: float64(3), int64(4), object(3)
memory usage: 7.8+ MB
